In [4]:
import pandas as pd

# S1.1.1 Target Compound Selection

To identify interesting small organic targets for our analysis, the EPA's 2020 Chemical Data Reporting must first be cleaned and filtered.

1. [EPA CDR Data Loading Loading](#epa-cdr-data-loading)
2. [Removing Entries Without CAS Numbers](#removing-entries-without-cas-numbers)
3. [Dropping Unnecessary Columns](#dropping-unnecessary-columns)
4. [Converting Production Volume to Numeric](#converting-production-volume-strings-to-values)
5. [Grouping Entries](#grouping-entries)
6. [Filtering out Aggregate Categories](#filtering-by-category)
7. [Exporting Data](#exporting-sorted-and-filtered-data)


#### EPA CDR Data Loading

In [12]:
df = pd.read_excel("2020 CDR Manufacture-Import Information.xlsx")
df.head()

,CHEMICAL REPORT ID,CHEMICAL NAME,CHEMICAL ID,CHEMICAL ID W/O DASHES,CHEMICAL ID TYPE,DOMESTIC PARENT COMPANY NAME,DOMESTIC PC ADDRESS LINE1,DOMESTIC PC ADDRESS LINE2,DOMESTIC PC CITY,DOMESTIC PC COUNTY / PARISH,...,PERCENT BYPRODUCT,WORKERS CODE,WORKERS,MAX CONC CODE,MAXIMUM CONCENTRATION,RECYCLED,PHYSICAL FORM(S),JOINT FC CODE,JOINT FUNCTION CATEGORY,JOINT FUNCT CAT OTHER DESC
0,711448,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849,Accession Number,CBI,CBI,CBI,CBI,CBI,...,0,W1,< 10,M3,30% – < 60%,No,Liquid,NaN,NaN,NaN
1,728903,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849,Accession Number,CBI,CBI,CBI,CBI,CBI,...,0,W4,50 – 99,M4,60% – < 90%,No,Liquid,NaN,NaN,NaN
2,702175,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849,Accession Number,CBI,CBI,CBI,CBI,CBI,...,CBI,W1,< 10,CBI,CBI,No,Liquid,NaN,NaN,NaN
3,704950,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",18849,18849,Accession Number,IDEMITSU LUBRICANTS AMERICA,701 PORT RD,NaN,JEFFERSONVILLE,Clark,...,0,W3,25 – 49,M2,1% – < 30%,No,Liquid,NaN,NaN,NaN
4,735410,"(Polyisobutenyl)dihydro-2,5-furandione reactio...",24103,24103,Accession Number,CBI,CBI,CBI,CBI,CBI,...,0,W5,100 – 499,M3,30% – < 60%,No,Liquid,NaN,NaN,NaN


#### Removing Entries Without CAS Numbers

In [13]:
original_length = len(df)
df_cas = df[df['CHEMICAL ID TYPE']=="CASRN"]
length_wo_CAS = len(df_cas)
pct_lost = -(length_wo_CAS-original_length)/original_length

print("% of entries lost: ", pct_lost)

% of entries lost:  0.019916745245049564


#### Dropping Unnecessary Columns

In [17]:
df2 = df_cas[["CHEMICAL ID", "CHEMICAL NAME", "2019 NATIONALLY AGGREGATED PV"]].copy()
df2.columns=["CAS", "Name", "Production"]

#### Converting Production Volume Strings to Values

The original dataframe provides production volume as a string with a range. Here, we split the ranges into low and high production estimates, clean up strings, and ensure they are numeric values.

In [19]:
df2[["Low Production", "High Production"]] = df2["Production"].astype(dtype=str).str.split("<",  expand=True)

df2.loc[:,"Low Production"] = df2["Low Production"].str.replace(",", "")
df2.loc[:,"Low Production"] = df2["Low Production"].str.replace("–", "")
df2.loc[:,"Low Production"] = df2["Low Production"].str.replace("≥", "")
df2.loc[:,"High Production"] = df2["High Production"].str.replace(",", "")

df2.loc[:,"Low Production"]=pd.to_numeric(df2["Low Production"])
df2.loc[:,"High Production"]=pd.to_numeric(df2["High Production"])

df2=df2.drop(columns="Production")
df2["Name"] = df2["Name"].str.lower()

df2.head()

,CAS,Name,Low Production,High Production
15,10016-20-3,.alpha.-cyclodextrin,NaN,1000000.0
16,128446-33-3,".alpha.-cyclodextrin, 2-hydroxypropyl ethers",NaN,1000000.0
17,128446-33-3,".alpha.-cyclodextrin, 2-hydroxypropyl ethers",NaN,1000000.0
18,56038-13-2,".alpha.-d-galactopyranoside, 1,6-dichloro-1,6-...",25187.0,NaN
19,99-20-7,".alpha.-d-glucopyranoside, .alpha.-d-glucopyra...",NaN,1000000.0


#### Grouping Entries

This current dataframe contains multiple entries per compound because the original dataset was intended to track every manufacturer or handler of the material. Now, we group them together, taking the mean of the Low Production and High Production columns across each group. In all cases, the production range was the same for every duplicate entry of a single compound because it was intended to represent aggregate annual production, so the average does not actually change the data.

In [23]:
df3_unfiltered=df2.groupby(["CAS", "Name"]).mean()

#### Filtering by Category

Several CAS entries refer to bulk aggregate categories. Because we intend to search for synthetic routes towards pure compounds, we filter out any of these categories based on keywords manually identified from the dataset. However, we do incldue the total mass of these categories when reporting the total percent of the annual chemical production volume our dataset represents.

In [25]:

lst = ["(petroleum)", "fuel", "hydrocarbons", "gasoline", "steelmaking", "residues", 'slags', 'coal']
for key in lst:
    df2 = df2[~df2["Name"].str.contains(key, case=False, regex=False)]

df3 = df2.groupby(["CAS", "Name"]).mean()

In [32]:
print('Total Mass, unfiltered dataset: ',df3_unfiltered.sum()["High Production"])
print('Total Mass, filtered dataset: ',df3.sum()["High Production"])
print('% of mass removed: ', (df3_unfiltered.sum()["High Production"]-df3.sum()["High Production"])/df3_unfiltered.sum()["High Production"]*100)

Total Mass, unfiltered dataset:  7364039850000.0
Total Mass, filtered dataset:  2579117250000.0
% of mass removed:  64.97686999887704


#### Exporting Sorted and Filtered Data

In [ ]:
df3=df3.sort_values("High Production", ascending=False)
df3.to_excel("../multicompound_data\CAS_to_production_volume.xlsx")